# Family River Crossing Puzzle

![](https://raw.githubusercontent.com/haskucy/family-river-cross-puzzle-analysis/main/family_river_crossing_puzzle_image.jpg)

Family River Crossing Puzzle is a game in which players work together to cross a river using a raft, following rules and minimizing trips. This repository analyzes the smallest number of moves and ways to solve the puzzle.



## Recreating The Game

In [45]:
from tqdm import tqdm

context_position = {0 : "Dad",
                    1 : "Son 1",
                    2 : "Son 2",
                    3 : "Mom",
                    4 : "Daugther 1",
                    5 : "Daugther 2",
                    6 : "Police",
                    7 : "Criminal",
                    8 : "Boat"}

# 0 means in island_1 (have not cross yet) 1 means in island_2 (has crossed)

example = "000000111" # means police, criminal and boat is in island_2

start = "000000000"
end = "111111111"

In [2]:
def check_allowed_position(string_position):
  """Checking if a position is allowed according to the rules"""

  #Mom and son without dad
  if (string_position[3] == string_position[1]) or (string_position[3] == string_position[2]):
    if (string_position[3] != string_position[0]):
      return False

  #Dad and daugther without mom
  if (string_position[0] == string_position[4]) or (string_position[0] == string_position[5]):
    if (string_position[0] != string_position[3]):
      return False

  #Criminal left alone with other member without police
  if (string_position[7] in string_position[0:6]):
    if (string_position[7] != string_position[6]):
      return False

  return True

def flip_bit(bit_string):
  """Flipping a bit"""

  if bit_string == "0": return "1"
  if bit_string == "1": return "0"
  print(f"Error Flip_bit {bit_string}")

def flip_position(string_position, position):
  """Flipping a bit from string_position at certain index""" 
  part1 = string_position[0:position]
  pinpoint = flip_bit(string_position[position])
  part2 = string_position[position+1:]
  return part1 + pinpoint + part2

def boat_check(position1, position2, person_position = "xxxxxxxx", boat_position = "x"):
  """Check if a boat can be navigated to other island (There is Dad, Mom, or Pol)""" 

  navigator_check = (position1 in [0,3,6]) or (position2 in [0,3,6])
  if person_position == "xxxxxxxx" and boat_position == "x":
    return navigator_check
  
  #checking if boat and person in the same island
  if (boat_position == person_position[position1]) and (boat_position == person_position[position2]):
    return navigator_check

  return False

def check_all_possible_next_move(string_position):
  """Check all next moves from a certain string_position""" 
  
  person_position = string_position[0:8]
  boat_position = string_position[8]
  next_boat_position = flip_bit(boat_position)
  list_next_move = []

  for i in range(len(person_position)):
    for j in range(len(person_position)):
      if not boat_check(i,j, person_position, boat_position): continue

      next_person_position = flip_position(person_position, i)
      next_person_position = flip_position(next_person_position, j) if i != j else next_person_position
      next_string_position_candidate = next_person_position + next_boat_position

      if check_allowed_position(next_string_position_candidate) and (next_string_position_candidate not in list_next_move):
        list_next_move.append(next_string_position_candidate)

  return list_next_move
      
#test
check_all_possible_next_move("000000000")

['100100001', '000000111']

In [3]:
context_position = {0 : "Dad",
                    1 : "S-A",
                    2 : "S-B",
                    3 : "Mom",
                    4 : "D-A",
                    5 : "D-B",
                    6 : "Pol",
                    7 : "Cri",
                    8 : "Boat"}

def string_visualization(string_position):
  """ Visualize string_position in a text format instead just in a bit format"""

  string_template = "(x0 x1 x2 x3 x4 x5 x6 x7 )x8________z8( z0 z1 z2 z3 z4 z5 z6 z7)"
  for i in range(len(string_position)):
    if string_position[i] == "0":
      viz_person_position = "x"+ str(i)
      viz_empty_position = "z"+ str(i)
    elif string_position[i] == "1":
      viz_person_position = "z"+ str(i)
      viz_empty_position = "x"+ str(i)
    string_template = string_template.replace(viz_person_position,context_position[i])
    string_template = string_template.replace(viz_empty_position, "___")
  return string_template

def check_differences(string_position1, string_position2):
  """ Check diff between two string position"""
  
  list_entity_moves = []
  for i in range(len(string_position1)):
    if string_position1[i] != string_position2[i]:
      list_entity_moves.append(context_position[i])
  
  string_template = ""
  for i in list_entity_moves:
    string_template = string_template + i + ", "
  
  result = string_template[0:-2] + " moves." if len(string_template) > 0 else "no one moves."
  return result


    
print(check_differences("000000000", "000000111"))
string_visualization("010001100")

Pol, Cri, Boat moves.


'(Dad ___ S-B Mom D-A ___ ___ Cri )Boat___________( ___ S-A ___ ___ ___ D-B Pol ___)'

## Tree Node Generation

In [5]:
class NodeBoat:
  def __init__(self, id, string_position, list_ancestor_id):
    self.id = id
    self.string_position = string_position
    self.list_ancestor_id = list_ancestor_id
    self.all_possible_move = check_all_possible_next_move(string_position)
  
  def __str__(self):
    return f"{self.id} \n{self.string_position} \n{self.list_ancestor_id} \n{self.all_possible_move}"
  
  def __repr__(self):
    return f"{self.string_position}"


start = NodeBoat("0", "000000000", [])

In [6]:
def map_nodeId_nodeBoat(node_id):
  return node_dict[node_id]

def map_node_stringPosition(node):
  return node.string_position

def map_nodeId_stringPosition(node_id):
  return map_node_stringPosition(map_nodeId_nodeBoat(node_id))

#Tree Generation
counter = 0
start = NodeBoat("0", "000000000", [])
node_dict = {"0" : start}
node_queue = [start]
while len(node_queue) != 0:
  current_node = node_queue.pop(0)
  list_all_next_moves = current_node.all_possible_move
  list_ancestor = current_node.list_ancestor_id.copy()
  list_ancestor.append(current_node.id)
  list_stringPosition_ancestor = list(map(map_nodeId_stringPosition, list_ancestor))
  
  for i in list_all_next_moves:
    if (i in list_stringPosition_ancestor): continue
    counter = counter + 1
    str_counter = str(counter)
    node_dict[str_counter] = NodeBoat(str_counter, i, list_ancestor)
    if (i != "111111111"): node_queue.append(node_dict[str_counter])

## All correct Answer and Analysis

In [44]:
list_of_correct_answer = []

#Printing The correct answer
for i in node_dict:
  if node_dict[i].string_position == "111111111":
    print("----start----")
    list_ancestor = node_dict[i].list_ancestor_id
    list_stringPosition_ancestor = list(map(map_nodeId_stringPosition, list_ancestor.copy()))


    counter = 0
    prev_moves = ""
    list_of_one_string_viz = []
    for k in list_stringPosition_ancestor:
      if counter != 0:
        print(check_differences(prev_moves, k))
      list_of_one_string_viz.append(f"{counter}. {string_visualization(k)}")
      print(f"{counter}. {string_visualization(k)}")
      print()
      prev_moves = k
      counter = counter + 1
    
    print(check_differences(prev_moves, '111111111'))
    print(f"{counter}. {string_visualization('111111111')}")
    list_of_one_string_viz.append(f"{counter}. {string_visualization('111111111')}")
    list_of_correct_answer.append(list_of_one_string_viz.copy())
    print()
    print(f"Total Moves = {len(list_stringPosition_ancestor)}")
    print()

----start----
0. (Dad S-A S-B Mom D-A D-B Pol Cri )Boat___________( ___ ___ ___ ___ ___ ___ ___ ___)

Pol, Cri, Boat moves.
1. (Dad S-A S-B Mom D-A D-B ___ ___ )___________Boat( ___ ___ ___ ___ ___ ___ Pol Cri)

Pol, Boat moves.
2. (Dad S-A S-B Mom D-A D-B Pol ___ )Boat___________( ___ ___ ___ ___ ___ ___ ___ Cri)

S-A, Pol, Boat moves.
3. (Dad ___ S-B Mom D-A D-B ___ ___ )___________Boat( ___ S-A ___ ___ ___ ___ Pol Cri)

Pol, Cri, Boat moves.
4. (Dad ___ S-B Mom D-A D-B Pol Cri )Boat___________( ___ S-A ___ ___ ___ ___ ___ ___)

Dad, S-B, Boat moves.
5. (___ ___ ___ Mom D-A D-B Pol Cri )___________Boat( Dad S-A S-B ___ ___ ___ ___ ___)

Dad, Boat moves.
6. (Dad ___ ___ Mom D-A D-B Pol Cri )Boat___________( ___ S-A S-B ___ ___ ___ ___ ___)

Dad, Mom, Boat moves.
7. (___ ___ ___ ___ D-A D-B Pol Cri )___________Boat( Dad S-A S-B Mom ___ ___ ___ ___)

Mom, Boat moves.
8. (___ ___ ___ Mom D-A D-B Pol Cri )Boat___________( Dad S-A S-B ___ ___ ___ ___ ___)

Pol, Cri, Boat moves.
9. (___ ___

In [14]:
list_of_correct_answer

[['0. (Dad S-A S-B Mom D-A D-B Pol Cri )Boat___________( ___ ___ ___ ___ ___ ___ ___ ___)',
  '1. (Dad S-A S-B Mom D-A D-B ___ ___ )___________Boat( ___ ___ ___ ___ ___ ___ Pol Cri)',
  '2. (Dad S-A S-B Mom D-A D-B Pol ___ )Boat___________( ___ ___ ___ ___ ___ ___ ___ Cri)',
  '3. (Dad ___ S-B Mom D-A D-B ___ ___ )___________Boat( ___ S-A ___ ___ ___ ___ Pol Cri)',
  '4. (Dad ___ S-B Mom D-A D-B Pol Cri )Boat___________( ___ S-A ___ ___ ___ ___ ___ ___)',
  '5. (___ ___ ___ Mom D-A D-B Pol Cri )___________Boat( Dad S-A S-B ___ ___ ___ ___ ___)',
  '6. (Dad ___ ___ Mom D-A D-B Pol Cri )Boat___________( ___ S-A S-B ___ ___ ___ ___ ___)',
  '7. (___ ___ ___ ___ D-A D-B Pol Cri )___________Boat( Dad S-A S-B Mom ___ ___ ___ ___)',
  '8. (___ ___ ___ Mom D-A D-B Pol Cri )Boat___________( Dad S-A S-B ___ ___ ___ ___ ___)',
  '9. (___ ___ ___ Mom D-A D-B ___ ___ )___________Boat( Dad S-A S-B ___ ___ ___ Pol Cri)',
  '10. (Dad ___ ___ Mom D-A D-B ___ ___ )Boat___________( ___ S-A S-B ___ ___ __

### Analysis Part [TODO]

In [43]:
# Melihat perbedaan dari tiap moves
# ubah k menjadi number of moves (dari 0 hingga 17)

k = 3
for i in list_of_correct_answer:
  print(i[k])

3. (Dad ___ S-B Mom D-A D-B ___ ___ )___________Boat( ___ S-A ___ ___ ___ ___ Pol Cri)
3. (Dad ___ S-B Mom D-A D-B ___ ___ )___________Boat( ___ S-A ___ ___ ___ ___ Pol Cri)
3. (Dad S-A ___ Mom D-A D-B ___ ___ )___________Boat( ___ ___ S-B ___ ___ ___ Pol Cri)
3. (Dad S-A ___ Mom D-A D-B ___ ___ )___________Boat( ___ ___ S-B ___ ___ ___ Pol Cri)
3. (Dad S-A S-B Mom ___ D-B ___ ___ )___________Boat( ___ ___ ___ ___ D-A ___ Pol Cri)
3. (Dad S-A S-B Mom ___ D-B ___ ___ )___________Boat( ___ ___ ___ ___ D-A ___ Pol Cri)
3. (Dad S-A S-B Mom D-A ___ ___ ___ )___________Boat( ___ ___ ___ ___ ___ D-B Pol Cri)
3. (Dad S-A S-B Mom D-A ___ ___ ___ )___________Boat( ___ ___ ___ ___ ___ D-B Pol Cri)


### Some insight:
* There are 8 different correct answer.
* Each of them differs on how the kids moves.
* Its a permutation on which kids moves first.



In [ ]:
# TODO: Continue Analysis.

## Tree Visualization

In [8]:
!pip install anytree
from anytree import Node, RenderTree

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.3 MB/s eta 0:00:00


In [9]:
def node_id_string_append(node_id):
  return node_dict[node_id].id + "-" + node_dict[node_id].string_position

root_anytree = Node(node_id_string_append("0"))
dict_anytree = {"0" : root_anytree}
for i in node_dict:
  if i == "0": continue
  current_node_parent_id = dict_anytree[node_dict[i].list_ancestor_id[-1]]
  dict_anytree[i] = Node(node_id_string_append(i), parent = current_node_parent_id)

for pre, fill, node in RenderTree(root_anytree):
    print("%s%s" % (pre, node.name))

0-000000000
├── 1-100100001
└── 2-000000111
    └── 3-000000010
        ├── 4-010000111
        │   └── 8-010000000
        │       ├── 12-111000001
        │       │   ├── 20-011000000
        │       │   │   ├── 28-111100001
        │       │   │   │   └── 44-111000000
        │       │   │   │       ├── 48-111110001
        │       │   │   │       │   └── 60-011010000
        │       │   │   │       │       └── 80-011010111
        │       │   │   │       ├── 49-111101001
        │       │   │   │       │   └── 61-011001000
        │       │   │   │       │       └── 81-011001111
        │       │   │   │       └── 50-111000111
        │       │   │   │           ├── 62-011000110
        │       │   │   │           │   └── 82-111100111
        │       │   │   │           │       ├── 100-111000110
        │       │   │   │           │       │   ├── 116-111110111
        │       │   │   │           │       │   │   ├── 124-011010110
        │       │   │   │           │       │   │   └